In [1]:
import pandas as pd
import mysql.connector
import numpy as np

In [2]:
# Define the CSV file path
csv_path = r"P:\Capstone Guvi\Redbus Data Scrapping\redbus_all_bus_data.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

## Data Cleaning

In [3]:
# Clean and process the 'Rating' column (remove "New", handle non-numeric values)
df["Rating"] = df["Rating"].astype(str).str.replace("New", "").str.strip().str.split().str[0]
df["Rating"] = pd.to_numeric(df["Rating"], errors='coerce').fillna(0)

In [4]:
# Replace NaN values in the DataFrame with suitable defaults
df.fillna({
    'Bus_Route': '',                 # Replace missing bus routes with an empty string
    'Bus_Link': '',                  # Replace missing bus links with an empty string
    'Bus_Name': '',                  # Replace missing bus names with an empty string
    'Bus_Type': '',                  # Replace missing bus types with an empty string
    'Departure_Time': '00:00:00',    # Replace missing times with default time
    'Arrival_Time': '00:00:00',      # Replace missing times with default time
    'Duration': '',                  # Replace missing duration with an empty string
    'Fare': 0,                       # Replace missing fares with 0
    'Rating': 0,                     # Replace missing ratings with 0
    'Seat_Availability': 0           # Replace missing seat availability with 0
}, inplace=True)

In [5]:
# Establish a connection to MySQL database
try:
    conn = mysql.connector.connect(
        host='localhost',
        user="root",
        password="12345",
        database="Redbus"
    )
    cursor = conn.cursor()

    # Define the insert query matching the column names from the MySQL table
    insert_query = '''
        INSERT INTO bus_routes (
            route_name, route_link, busname, bustype, departing_time, duration, 
            reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    # Prepare the data for insertion, ensuring column order matches the query
    data_to_insert = df[['Bus_Route', 'Bus_Link', 'Bus_Name', 'Bus_Type', 'Departure_Time', 
                         'Duration', 'Arrival_Time', 'Rating', 'Fare', 'Seat_Availability']].values.tolist()

    # Insert data into the MySQL table
    cursor.executemany(insert_query, data_to_insert)
    conn.commit()

    print("Data inserted successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

finally:
    cursor.close()
    conn.close()

Data inserted successfully
